<a href="https://colab.research.google.com/github/milkywayisouruniverse/Podcast_Listening_Time_Prediction-/blob/main/Podcast_Listening_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gc
import psutilfrom tqdm import tqdm
from sklearn.model_selection import KFold
import lightgbm as lgb
from category_encoders import TargetEncoder

In [ ]:
!pip install lightgbm

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train for prediction.csv to train for prediction.csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test for prediction.csv to test for prediction.csv


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving sample_submission.csv to sample_submission.csv


In [ ]:
import pandas as pd

train_df = pd.read_csv("train for prediction.csv")
train_df.head()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
0,0,Mystery Matters,Episode 98,NaN,True Crime,74.81,Thursday,Night,NaN,0.0,Positive,31.41998
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031


In [ ]:
import pandas as pd

train_df = pd.read_csv("test for prediction.csv")
train_df.head()


,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral


In [ ]:
import pandas as pd

train_df = pd.read_csv("sample_submission.csv")
train_df.head()


,id,Listening_Time_minutes
0,750000,45.437
1,750001,45.437
2,750002,45.437
3,750003,45.437
4,750004,45.437


In [ ]:
import pandas as pd

# Load the correct training data
train_df = pd.read_csv("train for prediction.csv")
test_df = pd.read_csv("test for prediction.csv") # Assuming you also have a separate test file

# Handle missing values
for col in ['Episode_Length_minutes', 'Guest_Popularity_percentage']:
    # Using train_df's median to fill missing values in both train_df and test_df
    train_df[col] = train_df[col].fillna(train_df[col].median())
    test_df[col] = test_df[col].fillna(train_df[col].median())

# Remove outliers and handle missing values for the training set
train_df = train_df.dropna()  # Drop rows with any missing values in train_df
train_df = train_df[train_df['Number_of_Ads'] < 10]  # Remove outliers in train_df

# Drop columns and create Episode_Num for both training and test sets
train_df['Episode_Num'] = train_df['Episode_Title'].str[8:]
test_df['Episode_Num'] = test_df['Episode_Title'].str[8:]
train_df.drop(columns=['Episode_Title'], inplace=True)
test_df.drop(columns=['Episode_Title'], inplace=True)

In [ ]:
from tqdm import tqdm # Import tqdm from the tqdm library
import pandas as pd
from itertools import combinations

# ... (your other code) ...

for r in pair_size:
    combos = list(combinations(combined_features, r))
    for cols in tqdm(combos):  # Now tqdm is defined and can be used
        col_name = '_'.join(cols)
        train_df[col_name] = train_df[list(cols)].astype(str).agg('_'.join, axis=1).astype('category')
        test_df[col_name] = test_df[list(cols)].astype(str).agg('_'.join, axis=1).astype('category')

100%|██████████| 45/45 [04:37<00:00,  6.16s/it]


In [ ]:
!pip install category_encoders # Correct pip install command
target = 'Listening_Time_minutes' # Define target variable before using it
X = train_df.drop(columns=[target])
y = train_df[target]

# Identify object and category columns
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

# Copy the datasets
X_encoded = X.copy()
X_test_encoded = test_df.copy()

# Apply Target Encoding
from category_encoders import TargetEncoder # Import TargetEncoder
encoder = TargetEncoder()
X_encoded[categorical_columns] = encoder.fit_transform(X[categorical_columns], y)
X_test_encoded[categorical_columns] = encoder.transform(X_test_encoded[categorical_columns])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
import lightgbm as lgb # Import the lightgbm module and assign it the alias 'lgb'
from sklearn.model_selection import KFold # Import KFold
# Ensure all columns are numeric
X_encoded = X_encoded.apply(pd.to_numeric)
X_test_encoded = X_test_encoded.apply(pd.to_numeric)

# Cross-validation training
cv = KFold(n_splits=5, shuffle=True, random_state=42)
fold_preds = []
models = []

for fold, (train_idx, valid_idx) in enumerate(cv.split(X_encoded)):
    X_train, X_valid = X_encoded.iloc[train_idx], X_encoded.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    model = lgb.LGBMRegressor( # Now 'lgb' is defined and can be used
        n_estimators=1000,
        learning_rate=0.03,
        num_leaves=1024,
        colsample_bytree=0.7,
        max_bin=1024,
        objective='l2',
        random_state=42
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        callbacks=[
            lgb.early_stopping(100),
            lgb.log_evaluation(100)
        ]
    )

    preds = model.predict(X_test_encoded)
    fold_preds.append(preds)
    models.append(model)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.627201 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 34013
[LightGBM] [Info] Number of data points in the train set: 599992, number of used features: 56
[LightGBM] [Info] Start training from score 45.462173
Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 11.0658
[200]	valid_0's l2: 8.61844
[300]	valid_0's l2: 8.46924
[400]	valid_0's l2: 8.40492
[500]	valid_0's l2: 8.36952


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import numpy as np

# ... (your other code) ...

# Load the sample submission file
sample_submission = pd.read_csv("sample_submission.csv")

final_preds = np.mean(fold_preds, axis=0)

submission = pd.DataFrame({
    'id': sample_submission['id'],
    'Listening_Time_minutes': final_preds
})
submission.to_csv("submission.csv", index=False)
submission.head()

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,id,Listening_Time_minutes
0,750000,NaN
1,750001,NaN
2,750002,NaN
3,750003,NaN
4,750004,NaN
